----
# 1 Data Preperation
## 1.1 read data to panda_dataframe

In [39]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import json

data_identification = pd.read_csv("data./data_identification.csv", skiprows=1, header=None, names=["tweet_id", "identification"])
emotion = pd.read_csv("data./emotion.csv", skiprows=1, header=None, names=["tweet_id", "emotion"])

tweets_DM_json=[]
for line in open('data./tweets_DM.json', 'r'):
    tweets_DM_json.append(json.loads(line))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
extract_tweets = [(tw['_source']['tweet']['tweet_id'],tw['_source']['tweet']['text'],tw['_source']['tweet']['hashtags'],tw['_score']) for tw in tweets_DM_json]

In [41]:
tweets_PD = pd.DataFrame(extract_tweets, columns=['tweet_id', 'text', 'hashtag','_score'])
tweets_PD

,tweet_id,text,hashtag,_score
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",[Snapchat],391
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...","[freepress, TrumpLegacy, CNN]",433
2,0x28b412,"Confident of your obedience, I write to you, k...",[bibleverse],232
3,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,[],376
4,0x2de201,"""Trust is not the same as faith. A friend is s...",[],989
...,...,...,...,...
1867530,0x316b80,When you buy the last 2 tickets remaining for ...,"[mixedfeeling, butimTHATperson]",827
1867531,0x29d0cb,I swear all this hard work gone pay off one da...,[],368
1867532,0x2a6a4f,@Parcel2Go no card left when I wasn't in so I ...,[],498
1867533,0x24faed,"Ah, corporate life, where you can date <LH> us...",[],840


In [42]:
train_data_identification_df = data_identification.loc[data_identification["identification"] == 'train']
train_data_identification_list = list(train_data_identification_df['tweet_id'])
test_data_identification_df = data_identification.loc[data_identification["identification"] == 'test']
test_data_identification_list = list(test_data_identification_df['tweet_id'])

In [43]:
train_dataset = pd.merge(tweets_PD, emotion, on='tweet_id')
ORI_train_dataset = pd.merge(tweets_PD, emotion, on='tweet_id')
train_dataset

,tweet_id,text,hashtag,_score,emotion
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",[Snapchat],391,anticipation
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...","[freepress, TrumpLegacy, CNN]",433,sadness
2,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,[],376,fear
3,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,"[authentic, LaughOutLoud]",120,joy
4,0x2c91a8,Still waiting on those supplies Liscus. <LH>,[],1021,anticipation
...,...,...,...,...,...
1455558,0x321566,I'm SO HAPPY!!! #NoWonder the name of this sho...,"[NoWonder, Happy]",94,joy
1455559,0x38959e,In every circumtance I'd like to be thankful t...,[],627,joy
1455560,0x2cbca6,there's currently two girls walking around the...,[blessyou],274,joy
1455561,0x24faed,"Ah, corporate life, where you can date <LH> us...",[],840,joy


In [44]:
test_dataset = pd.merge(test_data_identification_df, tweets_PD, on='tweet_id')
test_dataset = test_dataset.drop(['identification'], axis=1)
test_dataset

,tweet_id,text,hashtag,_score
0,0x28cc61,@Habbo I've seen two separate colours of the e...,[],107
1,0x2db41f,@FoxNews @KellyannePolls No serious self respe...,[],728
2,0x2466f6,"Looking for a new car, and it says 1 lady owne...",[womendrivers],491
3,0x23f9e9,@cineworld “only the brave” just out and fount...,[robbingmembers],28
4,0x1fb4e1,Felt like total dog 💩 going into open gym and ...,[],925
...,...,...,...,...
411967,0x2c4dc2,6 year old walks in astounded. Mum! Look how b...,[kids],792
411968,0x31be7c,Only one week to go until the #inspiringvolunt...,[inspiringvolunteerawards2017],34
411969,0x1ca58e,"I just got caught up with the manga for ""My He...",[],976
411970,0x35c8ba,Speak only when spoken to and make hot ass mus...,[],534


## 1.2 data cleaning

### 1.2.1 Mis-spelling data

Use some dataset on kaggle.


https://www.kaggle.com/datasets/bittlingmayer/spelling

In [45]:
misspell_data = pd.read_csv("data/aspell.txt", sep = ":", names = ["correction","misspell"])
misspell_data
miss_corr = dict(zip(misspell_data.misspell, misspell_data.correction))

In [46]:
def misspelled_correction(val):
    for x in val.split(): 
        if x in miss_corr.keys(): 
            val = val.replace(x, miss_corr[x]) 
    return val

train_dataset["text"] = train_dataset["text"].apply(lambda x : misspelled_correction(x))
train_dataset

,tweet_id,text,hashtag,_score,emotion
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",[Snapchat],391,anticipation
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...","[freepress, TrumpLegacy, CNN]",433,sadness
2,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,[],376,fear
3,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,"[authentic, LaughOutLoud]",120,joy
4,0x2c91a8,Still waiting on those supplies Liscus. <LH>,[],1021,anticipation
...,...,...,...,...,...
1455558,0x321566,I'm SO HAPPY!!! #NoWonder the name of this sho...,"[NoWonder, Happy]",94,joy
1455559,0x38959e,In every circumtance I'd like to be thankful t...,[],627,joy
1455560,0x2cbca6,there's currently two girls walking around the...,[blessyou],274,joy
1455561,0x24faed,"Ah, corporate life, where you can date <LH> us...",[],840,joy


### 1.2.2 Convert emoji to text
Use the dictionary from this website.

https://medium.com/geekculture/a-tutorial-on-performing-sentiment-analysis-in-python-3-using-the-natural-language-toolkit-nltk-40e5b35ab440

Install emoji to complete it

In [47]:
pip install emoji

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [48]:
def load_dict_smileys():
    return {
        ":‑)":"smiley",
        ":-]":"smiley",
        ":-3":"smiley",
        ":->":"smiley",
        "8-)":"smiley",
        ":-}":"smiley",
        ":)":"smiley",
        ":]":"smiley",
        ":3":"smiley",
        ":>":"smiley",
        "8)":"smiley",
        ":}":"smiley",
        ":o)":"smiley",
        ":c)":"smiley",
        ":^)":"smiley",
        "=]":"smiley",
        "=)":"smiley",
        ":-))":"smiley",
        ":‑D":"smiley",
        "8‑D":"smiley",
        "x‑D":"smiley",
        "X‑D":"smiley",
        ":D":"smiley",
        "8D":"smiley",
        "xD":"smiley",
        "XD":"smiley",
        ":‑(":"sad",
        ":‑c":"sad",
        ":‑<":"sad",
        ":‑[":"sad",
        ":(":"sad",
        ":c":"sad",
        ":<":"sad",
        ":[":"sad",
        ":-||":"sad",
        ">:[":"sad",
        ":{":"sad",
        ":@":"sad",
        ">:(":"sad",
        ":'‑(":"sad",
        ":'(":"sad",
        ":‑P":"playful",
        "X‑P":"playful",
        "x‑p":"playful",
        ":‑p":"playful",
        ":‑Þ":"playful",
        ":‑þ":"playful",
        ":‑b":"playful",
        ":P":"playful",
        "XP":"playful",
        "xp":"playful",
        ":p":"playful",
        ":Þ":"playful",
        ":þ":"playful",
        ":b":"playful",
        "<3":"love"
        }

In [49]:
# around 30s.
import emoji
def remove_emoticons(tweet):
    smilies = load_dict_smileys()
    split_tweet = tweet.split(" ")
    for key,val in smilies.items():
        if key in tweet:
            new_tweet = tweet.replace(key, val)
            tweet = new_tweet
            tweet = emoji.demojize(tweet)
    return tweet

train_dataset["text"] = train_dataset["text"].apply(lambda x: remove_emoticons(x))
train_dataset

,tweet_id,text,hashtag,_score,emotion
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",[Snapchat],391,anticipation
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...","[freepress, TrumpLegacy, CNN]",433,sadness
2,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,[],376,fear
3,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,"[authentic, LaughOutLoud]",120,joy
4,0x2c91a8,Still waiting on those supplies Liscus. <LH>,[],1021,anticipation
...,...,...,...,...,...
1455558,0x321566,I'm SO HAPPY!!! #NoWonder the name of this sho...,"[NoWonder, Happy]",94,joy
1455559,0x38959e,In every circumtance I'd like to be thankful t...,[],627,joy
1455560,0x2cbca6,there's currently two girls walking around the...,[blessyou],274,joy
1455561,0x24faed,"Ah, corporate life, where you can date <LH> us...",[],840,joy


### 1.2.3 Remove URLS
Install mysmallutils to complete it

In [50]:
pip install mysmallutils


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [51]:
# around 10s.
from mysutils.text import remove_urls
URL_PATTERN = r'[A-Za-z0-9]+://[A-Za-z0-9%-_]+(/[A-Za-z0-9%-_])*(#|\\?)[A-Za-z0-9%-_&=]*'
train_dataset["text"] = train_dataset["text"].apply(lambda x: remove_urls(x))
URL_PATTERN = r'[A-Za-z0-9%-_]+(/[A-Za-z0-9%-_])*(#|\\?)[A-Za-z0-9%-_&=]*'
train_dataset["text"] = train_dataset["text"].apply(lambda x: remove_urls(x))
train_dataset

,tweet_id,text,hashtag,_score,emotion
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",[Snapchat],391,anticipation
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...","[freepress, TrumpLegacy, CNN]",433,sadness
2,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,[],376,fear
3,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,"[authentic, LaughOutLoud]",120,joy
4,0x2c91a8,Still waiting on those supplies Liscus. <LH>,[],1021,anticipation
...,...,...,...,...,...
1455558,0x321566,I'm SO HAPPY!!! #NoWonder the name of this sho...,"[NoWonder, Happy]",94,joy
1455559,0x38959e,In every circumtance I'd like to be thankful t...,[],627,joy
1455560,0x2cbca6,there's currently two girls walking around the...,[blessyou],274,joy
1455561,0x24faed,"Ah, corporate life, where you can date <LH> us...",[],840,joy


### 1.2.4 Remove twitter mention
Use pandas.dataframe.str.replace to complete it

In [52]:
train_dataset['text'] = train_dataset['text'].str.replace(r'\s*@\w+', '', regex=True)
train_dataset['text'] = train_dataset['text'].str.replace(r'\s*\B@\w+', '', regex=True)
train_dataset['text'] = train_dataset['text'].str.replace(r'\s*@\S+', '', regex=True)
train_dataset['text'] = train_dataset['text'].str.replace(r'\s*@\S+\b', '', regex=True)
train_dataset

,tweet_id,text,hashtag,_score,emotion
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",[Snapchat],391,anticipation
1,0x2d5350,"As we see, Trump is dangerous to #freepress a...","[freepress, TrumpLegacy, CNN]",433,sadness
2,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,[],376,fear
3,0x1d755c,Thx for the BEST TIME tonight. What stories! ...,"[authentic, LaughOutLoud]",120,joy
4,0x2c91a8,Still waiting on those supplies Liscus. <LH>,[],1021,anticipation
...,...,...,...,...,...
1455558,0x321566,I'm SO HAPPY!!! #NoWonder the name of this sho...,"[NoWonder, Happy]",94,joy
1455559,0x38959e,In every circumtance I'd like to be thankful t...,[],627,joy
1455560,0x2cbca6,there's currently two girls walking around the...,[blessyou],274,joy
1455561,0x24faed,"Ah, corporate life, where you can date <LH> us...",[],840,joy


### 1.2.5 Remove twitter <<LH>>
Use pandas.dataframe.str.replace to complete it

In [53]:
train_dataset['text'] = train_dataset['text'].str.replace(r'<LH>', '', regex=True)
train_dataset

,tweet_id,text,hashtag,_score,emotion
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",[Snapchat],391,anticipation
1,0x2d5350,"As we see, Trump is dangerous to #freepress a...","[freepress, TrumpLegacy, CNN]",433,sadness
2,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂,[],376,fear
3,0x1d755c,Thx for the BEST TIME tonight. What stories! ...,"[authentic, LaughOutLoud]",120,joy
4,0x2c91a8,Still waiting on those supplies Liscus.,[],1021,anticipation
...,...,...,...,...,...
1455558,0x321566,I'm SO HAPPY!!! #NoWonder the name of this sho...,"[NoWonder, Happy]",94,joy
1455559,0x38959e,In every circumtance I'd like to be thankful t...,[],627,joy
1455560,0x2cbca6,there's currently two girls walking around the...,[blessyou],274,joy
1455561,0x24faed,"Ah, corporate life, where you can date using ...",[],840,joy


### 1.2.6 Remove Stopwords

Use the nltk stopwords to implement it

In [54]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')

train_dataset['text'] = train_dataset['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [55]:
from sklearn.model_selection import train_test_split

df_train, df_valid = train_test_split(train_dataset, test_size=0.2)
ORI_df_train, ORI_df_valid = train_test_split(ORI_train_dataset, test_size=0.2)

# 2 Data tokenize
There are 3 ways to tokenize the data

In [56]:
from sklearn.feature_extraction.text import CountVectorizer

BOW_500 = CountVectorizer(max_features=500, tokenizer=nltk.word_tokenize)
BOW_500.fit(df_train['text'])

X_train_BOW500 = BOW_500.transform(df_train['text'])
X_valid_BOW500 = BOW_500.transform(df_valid['text'])
y_train_BOW500 = df_train['emotion']
y_valid_BOW500 = df_valid['emotion']

test_bow500 = BOW_500.transform(test_dataset['text'])

C:\DM_hw2\venv\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [30]:
BOW_2000 = CountVectorizer(max_features=2000, tokenizer=nltk.word_tokenize)
BOW_2000.fit(df_train['text'])

X_train_BOW2000 = BOW_2000.transform(df_train['text'])
X_valid_BOW2000 = BOW_2000.transform(df_valid['text'])
y_train_BOW2000 = df_train['emotion']
y_valid_BOW2000 = df_valid['emotion']

test_bow2000 = BOW_2000.transform(test_dataset['text'])

C:\DM_hw2\venv\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [57]:
BOW_500 = CountVectorizer(max_features=500, tokenizer=nltk.word_tokenize)
BOW_500.fit(ORI_df_train['text'])

X_train_BOW500_ORI = BOW_500.transform(ORI_df_train['text'])
X_valid_BOW500_ORI = BOW_500.transform(ORI_df_valid['text'])
y_train_BOW500_ORI = ORI_df_train['emotion']
y_valid_BOW500_ORI = ORI_df_valid['emotion']

test_bow500_ORI = BOW_500.transform(test_dataset['text'])

In [58]:
BOW_2000 = CountVectorizer(max_features=2000, tokenizer=nltk.word_tokenize)
BOW_2000.fit(ORI_df_train['text'])

X_train_BOW2000_ORI = BOW_2000.transform(ORI_df_train['text'])
X_valid_BOW2000_ORI = BOW_2000.transform(ORI_df_valid['text'])
y_train_BOW2000_ORI = ORI_df_train['emotion']
y_valid_BOW2000_ORI = ORI_df_valid['emotion']

test_bow2000_ORI = BOW_2000.transform(test_dataset['text'])

In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer
# from nltk.corpus import stopwords
# twt_tknzr = nltk.tokenize.TweetTokenizer(strip_handles=True, reduce_len=True)
# tfidf_vector = TfidfVectorizer(max_features=500, tokenizer=twt_tknzr.tokenize)
tfidf_vector = TfidfVectorizer(max_features=500) 

tfidf_vector.fit(ORI_df_train['text'])
X_train_tfidf_ORI = tfidf_vector.fit_transform(ORI_df_train['text'])
X_valid_tfidf_ORI = tfidf_vector.fit_transform(ORI_df_valid['text'])
y_train_tfidf_ORI = ORI_df_train['emotion']
y_valid_tfidf_ORI = ORI_df_valid['emotion']

test_tfidf_tfidf_ORI = tfidf_vector.transform(test_dataset['text'])

In [60]:
tfidf_vector = TfidfVectorizer(max_features=500) 

tfidf_vector.fit(ORI_df_train['text'])
X_train_tfidf = tfidf_vector.fit_transform(df_train['text'])
X_valid_tfidf = tfidf_vector.fit_transform(df_valid['text'])
y_train_tfidf = df_train['emotion']
y_valid_tfidf = df_valid['emotion']

test_tfidf_tfidf = tfidf_vector.transform(test_dataset['text'])

In [61]:
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = keras.preprocessing.text.Tokenizer(num_words=10000, oov_token='<UNK>')
tokenizer.fit_on_texts(df_train['text'].tolist())
tokenizer.fit_on_texts(df_valid['text'].tolist())
def get_sequences(tokenizer, tweets):
    sequences = tokenizer.texts_to_sequences(tweets)
    padded_sequences = pad_sequences(sequences, truncating='post', maxlen=100, padding='post')
    return padded_sequences

X_train_keras = get_sequences(tokenizer, df_train['text'].tolist())
X_valid_keras = get_sequences(tokenizer, df_valid['text'].tolist())

emotions = set(df_train['emotion'].tolist())
emotions_to_index = dict((c, i) for i, c in enumerate(emotions))
index_to_emotions = dict((v, k) for k, v in emotions_to_index.items())
names_to_ids = lambda labels: np.array([emotions_to_index.get(x) for x in labels])

y_train_keras = names_to_ids(df_train['emotion'].tolist())
y_valid_keras = names_to_ids(df_valid['emotion'].tolist())

In [36]:
tokenizer = keras.preprocessing.text.Tokenizer(num_words=10000, oov_token='<UNK>')
tokenizer.fit_on_texts(ORI_df_train['text'].tolist())
tokenizer.fit_on_texts(ORI_df_valid['text'].tolist())
def get_sequences(tokenizer, tweets):
    sequences = tokenizer.texts_to_sequences(tweets)
    padded_sequences = pad_sequences(sequences, truncating='post', maxlen=100, padding='post')
    return padded_sequences

X_train_keras_ORI = get_sequences(tokenizer, ORI_df_train['text'].tolist())
X_valid_keras_ORI = get_sequences(tokenizer, ORI_df_valid['text'].tolist())

emotions = set(df_train['emotion'].tolist())
emotions_to_index = dict((c, i) for i, c in enumerate(emotions))
index_to_emotions = dict((v, k) for k, v in emotions_to_index.items())
names_to_ids = lambda labels: np.array([emotions_to_index.get(x) for x in labels])

y_train_keras_ORI = names_to_ids(ORI_df_train['emotion'].tolist())
y_valid_keras_ORI = names_to_ids(ORI_df_valid['emotion'].tolist())

In [62]:
X_test_keras = get_sequences(tokenizer, test_dataset['text'].tolist())

# 3 Train different models to predict
## 3.1 DecisionTree

In [63]:
#about 50mins
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
model = DecisionTreeClassifier(random_state=1012)
model = model.fit(X_train_BOW500, y_train_BOW500)
y_train_pred = model.predict(X_train_BOW500)
y_test_pred = model.predict(X_valid_BOW500)

accuracy = accuracy_score(y_true=y_train_BOW500, y_pred=y_train_pred)
print("train accuracy is ", accuracy)

accuracy = accuracy_score(y_true=y_valid_BOW500, y_pred=y_test_pred)
print("valid accuracy is ", accuracy)

train accuracy is  0.9017965563141397
valid accuracy is  0.39223943966775787


In [64]:
test_bow500_result = model.predict(test_bow500)
test_bow500_result

array(['sadness', 'joy', 'joy', ..., 'fear', 'anger', 'sadness'],
      dtype=object)

In [65]:
pd.DataFrame(test_dataset.iloc[:,0]).to_csv("INDEX_01_DT_BOW500.csv")
pd.DataFrame(test_bow500_result).to_csv("01_DT_BOW500.csv")

## 3.2 Random Forest

In [66]:
from sklearn.ensemble import RandomForestClassifier

model_RFC = RandomForestClassifier(bootstrap=True, n_estimators=1000, max_depth=4, random_state=1012)
model_RFC.fit(X_train_BOW500, y_train_BOW500)
y_train_pred = model_RFC.predict(X_train_BOW500)
y_test_pred = model_RFC.predict(X_valid_BOW500)

accuracy = accuracy_score(y_true=y_train_BOW500, y_pred=y_train_pred)
print("train accuracy is ", accuracy)

accuracy = accuracy_score(y_true=y_valid_BOW500, y_pred=y_test_pred)
print("valid accuracy is ", accuracy)

train accuracy is  0.3660732534673022
valid accuracy is  0.3659438087615462


In [67]:
test_RFC_result = model_RFC.predict(test_bow500)
pd.DataFrame(test_dataset.iloc[:,0]).to_csv("INDEX_02_RF_BOW500.csv")
pd.DataFrame(test_RFC_result).to_csv("02_RF_BOW500.csv")

## 3.3 MultinomialNB

In [68]:
from sklearn.naive_bayes import MultinomialNB

MNB = MultinomialNB()
MNB.fit(X_train_tfidf, y_train_tfidf)
y_train_pred = MNB.predict(X_train_tfidf)
y_test_pred = MNB.predict(X_valid_tfidf)

accuracy = accuracy_score(y_true=y_train_tfidf, y_pred=y_train_pred)
print("train accuracy is ", accuracy)

accuracy = accuracy_score(y_true=y_valid_tfidf, y_pred=y_test_pred)
print("valid accuracy is ", accuracy)

train accuracy is  0.4344815148782687
valid accuracy is  0.3232043914218877


In [69]:
test_MNB_result = MNB.predict(test_tfidf_tfidf)
pd.DataFrame(test_dataset.iloc[:,0]).to_csv("INDEX_03_MNB_TFIDF500.csv")
pd.DataFrame(test_MNB_result).to_csv("03_MNB_TFIDF500.csv")

In [70]:
MNB_tfidf = MultinomialNB()
MNB_tfidf.fit(X_train_tfidf_ORI, y_train_tfidf_ORI)
y_train_pred = MNB_tfidf.predict(X_train_tfidf_ORI)
y_test_pred = MNB_tfidf.predict(X_valid_tfidf_ORI)

accuracy = accuracy_score(y_true=y_train_tfidf_ORI, y_pred=y_train_pred)
print("train accuracy is ", accuracy)

accuracy = accuracy_score(y_true=y_valid_tfidf_ORI, y_pred=y_test_pred)
print("valid accuracy is ", accuracy)

train accuracy is  0.436769290222852
valid accuracy is  0.34386647109541657


In [71]:
test_MNB_result = MNB_tfidf.predict(test_tfidf_tfidf_ORI)
pd.DataFrame(test_dataset.iloc[:,0]).to_csv("INDEX_04_MNB_TFIDF500_ORI.csv")
pd.DataFrame(test_MNB_result).to_csv("04_MNB_TFIDF500_ORI.csv")

## 3.4  XGBClassifier

In [76]:
pip install xgboost


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 99.8/99.8 MB 168.2 kB/s eta 0:00:00
     -------------------------------------- 99.8/99.8 MB 615.4 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [77]:
# about 102mins
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train_tfidf_label = le.fit_transform(y_train_tfidf)

model_XGB_tfidf = XGBClassifier(n_estimators=1500, learning_rate= 0.05, max_depth=5, num_class=8)
model_XGB_tfidf.fit(X_train_tfidf, y_train_tfidf_label)
y_train_pred = model_XGB_tfidf.predict(X_train_tfidf)
y_test_pred = model_XGB_tfidf.predict(X_valid_tfidf)

accuracy = accuracy_score(y_true=y_train_tfidf_label, y_pred=y_train_pred)
print("train accuracy is ", accuracy)

accuracy = accuracy_score(y_true=y_valid_tfidf, y_pred=y_test_pred)
print("valid accuracy is ", accuracy)

train accuracy is  0.47113572931426856
valid accuracy is  0.0


In [ ]:
# tree=1000, train accuracy is 0.455, valid is unavaliable, about 102 mins.
# tree=1500, train accuracy is 0.455, valid is unavaliable, about 130 mins.

In [78]:
# [0 1 2 3 4 5 6 7], got ['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise' 'trust']
int_to_emotion = {'0':'anger','1':'anticipation','2':'disgust','3':'fear','4':'joy','5':'sadness','6':'surprise','7':'trust'}
test_XGB_tfidf_result = model_XGB_tfidf.predict(test_tfidf_tfidf)
pd.DataFrame(test_dataset.iloc[:,0]).to_csv("INDEX_05_XGB_tfidf.csv")
pd.DataFrame(test_XGB_tfidf_result).to_csv("05_XGB_tfidf.csv")
